In [14]:
import nrrd
import numpy as np
import pandas as pd
import pyvista as pv
from skimage.morphology import area_closing

In [9]:
folder_path = './pre_lung_segmentations/'

In [6]:
file_name = 'Lung segmentation-AI-10.seg.nrrd'

In [10]:
data, header = nrrd.read( folder_path + file_name )

In [11]:
masks = {}
for i in range(data.shape[0]):
    seg_id = header.get(f"Segment{i}_ID")
    label_val = int(header.get(f"Segment{i}_LabelValue"))
    mask = data[i] == label_val
    masks[seg_id] = mask

In [12]:
### Marching Cubes

In [15]:
# Volume data
lungs_data = masks['lungs']
filled = area_closing(lungs_data, area_threshold=10, connectivity=3)
# fills the 1-voxel and 8-voxel cavities (threshold is in voxels)

In [16]:
def view_boolean_mask(mask: np.ndarray,
                      spacing=(1.0, 1.0, 1.0),
                      origin=(0.0, 0.0, 0.0),
                      mode="surface",   # "surface" or "volume"
                      iso=0.5):
    """
    Visualize a 3D boolean mask with PyVista.
    Assumes mask shape is (z, y, x). If yours is (x, y, z), set transpose=False below.
    """
    assert mask.ndim == 3, f"Expected 3D, got {mask.shape}"
    # Convert to uint8: False->0 (empty), True->1 (filled)
    scalars_zyx = mask.astype(np.uint8)

    # VTK expects (x, y, z) with Fortran-order ravel; transpose from (z,y,x) -> (x,y,z)
    vol_xyz = np.transpose(scalars_zyx, (2, 1, 0))
    nx, ny, nz = vol_xyz.shape

    img = pv.ImageData()
    img.dimensions = (nx, ny, nz)
    img.origin = origin          # physical location of (0,0,0)
    img.spacing = spacing        # voxel size (dx, dy, dz)
    img.point_data["labels"] = vol_xyz.ravel(order="F")

    pl = pv.Plotter(window_size=(1000, 800))
    if mode == "volume":
        # Make zeros transparent, ones visible
        # (keep it simple; adjust if too faint/opaque)
        opacity = [0.0, 0.0, 0.2, 0.6, 1.0]
        pl.add_volume(img, scalars="labels", shade=False, opacity=opacity)
    else:
        
        print("this happened!")
        # Extract a surface around the filled region
        surf = img.contour([iso], method='marching_cubes').compute_normals(
            auto_orient_normals=True, feature_angle=60.0
        )
        pl.add_mesh(surf, opacity=1.0, specular=0.15)
        
        arrow_len = 0.01 * np.linalg.norm(surf.length)
        arrows = surf.glyph(orient="Normals", scale=False, factor=arrow_len)
        #pl.add_mesh(arrows, color="tomato")

    pl.add_axes(); pl.show_grid()
    pl.camera.zoom(1.2)
    pl.show()

# Example:
mask = lungs_data
#view_boolean_mask(mask, spacing=(1,1,1), mode="surface")
view_boolean_mask(mask, mode="volume")

Widget(value='<iframe src="http://localhost:43459/index.html?ui=P_0x731f3c7260c0_0&reconnect=auto" class="pyvi…